In [ ]:
import pandas as pd
import json
import requests
import math
import numpy as np
import urllib

In [ ]:
import utilcalcs as calc
import geo_agg as geo
from censusAPI import myAPI
from county_codes import stco

In [ ]:
mdhhinc = {'mdhhiu10': [0, 9999],
 'mdhhi10t14': [10000, 14999],
 'mdhhi15t19': [15000, 19999],
 'mdhhi20t24': [20000, 24999],
 'mdhhi25t29': [25000, 29999],
 'mdhhi30t34': [30000, 34999],
 'mdhhi35t39': [35000, 39999],
 'mdhhi40t44': [40000, 44999],
 'mdhhi45t49': [45000, 49999],
 'mdhhi50t59': [50000, 59999],
 'mdhhi60t74': [60000, 74999],
 'mdhhi75t99': [75000, 99999],
 'mdhi100t124': [100000, 124999],
 'mdhi125t149': [125000, 149999],
 'mdhi150t199': [150000, 199999],
 'mdhhi200pl': [200000, 9999999]}

In [ ]:
#My search parameters
y1 = '2018'
y0 = '2010'

cols_1 = f'group(DP03)'
cols_2 = f'group(DP02)'

#bsource = 'acs/acs5'
dsource = 'acs/acs5/profile'

In [ ]:
def get_data(year,source,cols):
    url = f"https://api.census.gov/data/{year}/{source}?get={cols}&for=county:*&in=state:*&key={myAPI}"
    resp = requests.request('GET', url).content
    df_co = pd.DataFrame(json.loads(resp)[1:])
    df_co.columns = json.loads(resp)[0]

    url = f"https://api.census.gov/data/{year}/{source}?get={cols}&for=us:*&key={myAPI}"
    resp = requests.request('GET', url).content
    df_us = pd.DataFrame(json.loads(resp)[1:])
    df_us.columns = json.loads(resp)[0]

    df = pd.concat([df_co,df_us],sort=True)
    return df

def clean_data(df,var):
    dff = df[var].copy()
    var_num = var[1:]
    for col in var_num:
        dff[col] = dff[col].astype(float)
    dff = dff.replace([999999999, 555555555, 333333333, 222222222,\
                    666666666, 888888888, -999999999, -555555555,\
                    -333333333, -222222222, -666666666, -888888888], 0)
    return dff

In [ ]:
#Median functions
def get_median(buckets, row):
    ordered = list(buckets.keys())
    orderedE = [i+'e' for i in ordered]
    N = row[orderedE].sum()
    C = 0
    i = 0
    while C <= N/2 and i<=len(buckets.keys())-1:
        C += int(row[orderedE[i]])
        i += 1
    i = i-1
    if i == 0:
        median = list(buckets.values())[0][1]
    elif C == 0: 
        median =0
    elif i == len(buckets.keys())-1:
        median = list(buckets.values())[-1][1]
    else: 
        C = C - int(row[orderedE[i]])
        L = buckets[ordered[i]][0]
        F = int(row[orderedE[i]])
        W = buckets[ordered[i]][1] - buckets[ordered[i]][0]
        median = L + (N/2 - C)*W/F
    return median

def get_median_moe(buckets, row, DF=1.1):
    ordered = list(buckets.keys())
    orderedE = [i+'e' for i in ordered]
    B = row[orderedE].sum()
    if B == 0: 
        return np.nan
    else:
        cumm_dist = list(np.cumsum(row[orderedE])/B*100)

        se_50 = DF*(((93/(7*B))*2500))**0.5
        
        if se_50 >= 50:
            return np.nan
        else: 
            p_lower = 50 - se_50
            p_upper = 50 + se_50
            
            lower_bin = min([cumm_dist.index(i) for i in cumm_dist if i > p_lower])
            upper_bin = min([cumm_dist.index(i) for i in cumm_dist if i > p_upper])
            
            if lower_bin >= len(ordered)-1 or upper_bin >= len(ordered)-1:
                return np.nan
            else:
                if lower_bin == upper_bin:
                    A1 = min(buckets[ordered[lower_bin]])
                    A2 = min(buckets[ordered[lower_bin+1]])
                    C1 = cumm_dist[lower_bin-1]
                    C2 = cumm_dist[lower_bin]
                    lowerbound = (p_lower - C1)*(A2-A1)/(C2-C1) + A1 
                    upperbound = (p_upper - C1)*(A2-A1)/(C2-C1) + A1

                else:
                    A1_l = min(buckets[ordered[lower_bin]])
                    A2_l = min(buckets[ordered[lower_bin+1]])
                    C1_l = cumm_dist[lower_bin-1]
                    C2_l = cumm_dist[lower_bin]

                    A1_u = min(buckets[ordered[upper_bin]])
                    A2_u = min(buckets[ordered[upper_bin+1]])
                    C1_u = cumm_dist[upper_bin-1]
                    C2_u = cumm_dist[upper_bin]

                    lowerbound = (p_lower - C1_l)*(A2_l-A1_l)/(C2_l-C1_l) + A1_l 
                    upperbound = (p_upper - C1_u)*(A2_u-A1_u)/(C2_u-C1_u) + A1_u

                return (upperbound - lowerbound)*1.645/2